First we import the libraries and load the training data

In [69]:
import numpy as np
from scipy.sparse import csr_matrix,rand,coo_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import numpy.linalg as LA
import time
import sys
import math
from subprocess import call

path = "preprocess_train_matrix_fact"

with open(path) as train:
    rows = []
    cols = []
    data = []
    max_row = 0
    max_col = 0
    for line in train.readlines():
        current = line.split(" ")
        r = int(current[0])
        if r > max_row:
            max_row = r
        for movie in current[1].split(","):
            movie_list = movie.split("#")
            rows.append(r)
            c = int(movie_list[0])
            cols.append(c)
            if c > max_col:
                max_col = c
            data.append(float(movie_list[1]))

    rating_matrix =csr_matrix((data,(rows,cols)), shape=(max_row+1,max_col+1))
    
print rating_matrix.data.shape

(820367,)


We train pmf and get k latent factors

In [156]:
from pmf import matrix_fact
k = 100
U = np.random.random((rating_matrix.shape[0],k))

V = np.random.random((rating_matrix.shape[1],k))

s = time.time()

E,_,U,V = matrix_fact(rating_matrix, U, V, k,alpha=0.005,lambda_u = 0.01, lambda_v = 0.01)
rating_matrix.data = rating_matrix.data + 3
e = time.time()

Lambda_U = 0.010000
Lambda_V = 0.010000
Initial error: 250668380.54
Epoch 1
Cost: 1009652.15
Epoch 2
Cost: 643626.86
Epoch 3
Cost: 504731.70
Epoch 4
Cost: 422721.02
Epoch 5
Cost: 369979.49
Epoch 6
Cost: 334376.65
Epoch 7
Cost: 308516.52
Epoch 8
Cost: 288488.77
Epoch 9
Cost: 272202.46
Epoch 10
Cost: 258465.46
Epoch 11
Cost: 246555.04
Epoch 12
Cost: 236009.85
Epoch 13
Cost: 226523.03
Epoch 14
Cost: 217883.95
Epoch 15
Cost: 209944.06
Epoch 16
Cost: 202595.82
Epoch 17
Cost: 195759.19
Epoch 18
Cost: 189372.90
Epoch 19
Cost: 183388.71
Epoch 20
Cost: 177767.61
Epoch 21
Cost: 172477.28
Epoch 22
Cost: 167490.36
Epoch 23
Cost: 162783.23
Epoch 24
Cost: 158335.21
Epoch 25
Cost: 154127.90
Epoch 26
Cost: 150144.77
Epoch 27
Cost: 146370.87
Epoch 28
Cost: 142792.52
Epoch 29
Cost: 139397.20
Epoch 30
Cost: 136173.37
Epoch 31
Cost: 133110.33
Epoch 32
Cost: 130198.22
Epoch 33
Cost: 127427.82
Epoch 34
Cost: 124790.60
Epoch 35
Cost: 122278.59
Epoch 36
Cost: 119884.38
Epoch 37
Cost: 117601.02
Epoch 38
Cost: 

In [157]:
user_features = {}
counter = 0
extreme_rating = rating_matrix.copy()
for i in range(len(extreme_rating.data)):
    if extreme_rating.data[i] > 1 and  extreme_rating.data[i] < 5:
        extreme_rating.data[i] = 0

extreme_rating.eliminate_zeros()


extreme_rating= extreme_rating.toarray()

We create the features from the pmf results and get the statistics

In [158]:
X_train = []
Y_train = []
s = time.time()

for u in range(extreme_rating.shape[0]):
    user_features[u] = []
    current = extreme_rating[u,:]
    items = current.nonzero()[0]


    mult = np.multiply(U[u,:],V)
    
    flag = False
    for i in items:
        x_ui = mult[i]
        for j in items:
            if i == j:
                continue

            if abs(extreme_rating[u,i] - extreme_rating[u,j]) >= 4:
                x_uj = mult[j]
                features = x_ui - x_uj
                X_train.append(features)
                label = (extreme_rating[u,i] - extreme_rating[u,j])/abs(extreme_rating[u,i] - extreme_rating[u,j])
                Y_train.append(label)
                user_features[u].append([features,label])
                counter +=1
e = time.time()



print counter

3520432


In [119]:
print e-s

29.9132840633


In [120]:
pos = 0.0
neg = 0.0

for u in user_features:
    for r in user_features[u]:
        if r[1] > 0:
            pos+=1
        else:
            neg+=1
print pos/neg

1.0


In [121]:
len(user_features[1234])

196

In [122]:
len(user_features[4321])

64

We persist features to disk to train SVM

In [123]:
# with open("training_set","a") as f:
#     for u in user_features:
#         for r in user_features[u]:
#             string = ""
#             string+= str(int(r[1]))
#             for idx,feature in enumerate(r[0]):
#                 string+=" " + str(idx+1) + ":" +str(feature)
#             string += "\n"
#             f.write(string)

            
            

We train SVM and LR models

In [124]:
# s = time.time()

# call(["liblinear-2.1/train", "training_set"])

# e = time.time()

# print "SVM took %2.f seconds"%(e-s)

In [125]:
# with open("training_set.model") as f:
#     lines = f.readlines()
#     i = 0
#     while lines[i] != "w\n":
#         i+=1
#     i+=1
#     SVM_W = []
#     while i < len(lines):
#         SVM_W.append(float(lines[i]))
#         i+=1


In [159]:
from logit import train_lr,predict_lr
learning_rate = 10**-5
lambda_value = 0.1
s = time.time()

X_train = np.array(X_train)
W = np.zeros((X_train.shape[1], len(set(Y_train)))).astype("float64")
lr_W = train_lr(X_train,Y_train,W,100)

e = time.time()
print "LR took %2.f seconds"%(e-s)

Initial likelihood is -2440177.5152
Train Accuracy: 0.50000
Epoch 0 likelihood is -645475.8923
Train Accuracy: 0.99941
Epoch 1 likelihood is -374530.0002
Train Accuracy: 0.99949
Epoch 2 likelihood is -269174.0837
Train Accuracy: 0.99953
Epoch 3 likelihood is -212970.5683
Train Accuracy: 0.99955
Epoch 4 likelihood is -177889.7140
Train Accuracy: 0.99957
Epoch 5 likelihood is -153831.2483
Train Accuracy: 0.99958
Epoch 6 likelihood is -136262.8328
Train Accuracy: 0.99959
Epoch 7 likelihood is -122845.1829
Train Accuracy: 0.99960
Epoch 8 likelihood is -112247.0866
Train Accuracy: 0.99960
Epoch 9 likelihood is -103654.2572
Train Accuracy: 0.99961
Epoch 10 likelihood is -96540.0845
Train Accuracy: 0.99961
Epoch 11 likelihood is -90548.5838
Train Accuracy: 0.99961
Epoch 12 likelihood is -85430.2722
Train Accuracy: 0.99962
Epoch 13 likelihood is -81004.9865
Train Accuracy: 0.99962
Epoch 14 likelihood is -77139.2887
Train Accuracy: 0.99963
Epoch 15 likelihood is -73732.1857
Train Accuracy: 0.99

In [160]:
dev_X = []
with open ("svm_pred","a") as svm_pred:
    with open("lr_pred","a") as lr_pred:
        with open("dev.csv") as dev:
            for line in dev.readlines():
                line = line.split(",")
                u = int(line[1])
                v = int(line[0])
                features = np.multiply(U[u,:],V[v,:])
#                 svm_pred.write(str(np.dot(SVM_W,features)) + "\n")
                lr_pred.write(str(np.dot(lr_W,features))+"\n")

In [161]:
with open("predictions.txt","a") as pred:
    with open("test.csv") as dev:
            for line in dev.readlines():
                line = line.split(",")
                u = int(line[1])
                v = int(line[0])
                features = np.multiply(U[u,:],V[v,:])
                pred.write(str(np.dot(lr_W,features))+"\n")